In [2]:
import sys
sys.path.append(r'E:\project\backtrader')

In [3]:
import backtrader as bt
import backtrader.indicators as btind
import backtrader.feeds as btfeeds

# Data Feeds - Passing them around

The basis of the work with the platform will be done with Strategies. And these will get passed Data Feeds. The platform end user does not need to care about receiving them:

Data Feeds are automagically provided member variables to the strategy in the form of an array and shortcuts to the array positions



Quick preview of a Strategy derived class declaration and running the platform:

```python
class MyStrategy(bt.Strategy):
    params = dict(period=20)

    def __init__(self):

        sma = btind.SimpleMovingAverage(self.datas[0], period=self.params.period)

    ...

cerebro = bt.Cerebro()

...

data = btfeeds.MyFeed(...)
cerebro.adddata(data)

...

cerebro.addstrategy(MyStrategy, period=30)
```

# Omitting the Data Feeds
The example above can be further simplified to:

```python
    params = dict(period=20)

    def __init__(self):

        sma = btind.SimpleMovingAverage(period=self.params.period)

    ...
```

self.data has been completely removed from the invocation of SimpleMovingAverage. If this is done, the indicator (in this case the SimpleMovingAverage) receives the first data of the object in which is being created (the Strategy), which is self.data (aka self.data0 or self.datas[0])

# Almost everything is a Data Feed

Not only Data Feeds are data and can be passed around. Indicators and results of Operations are also data.

In the previous example the SimpleMovingAverage was receiving self.datas[0] as input to operate on. An example with operations and extra indicators:

In [8]:
class MyStrategy(bt.Strategy):
    params = dict(period1=20, period2=25, period3=10, period4=20)

    def __init__(self):

        sma1 = btind.SimpleMovingAverage(self.datas[0], period=self.p.period1)

        # This 2nd Moving Average operates using sma1 as "data"
        sma2 = btind.SimpleMovingAverage(sma1, period=self.p.period2)

        # New data created via arithmetic operation
        something = sma2 - sma1 + self.data.close

        # This 3rd Moving Average operates using something  as "data"
        sma3 = btind.SimpleMovingAverage(something, period=self.p.period3)

        # Comparison operators work too ...
        greater = sma3 > sma1

        # Pointless Moving Average of True/False values but valid
        # This 4th Moving Average operates using greater  as "data"
        sma3 = btind.SimpleMovingAverage(greater, period=self.p.period4)


Basically everything gets transformed into an object which can be used as a data feed once it has been operated upon.



# Parameters

Mostly every other class in the platform supports the notion of parameters.

Parameters along with default values are declared as a class attribute (tuple of tuples or dict-like object)

Keywords args (**kwargs) are scanned for matching parameters, removing them from **kwargs if found and assigning the value to the corresponding parameter

And parameters can be finally used in instances of the class by accessing the member variable self.params (shorthand: self.p)

The previous quick Strategy preview already contains a parameters example, but for the sake of redundancy, again, focusing only on the parameters. Using tuples:

The previous quick Strategy preview already contains a parameters example, but for the sake of redundancy, again, focusing only on the parameters. Using tuples:

In [10]:
class MyStrategy(bt.Strategy):
    params = (('period', 20),)

    def __init__(self):
        sma = btind.SimpleMovingAverage(self.data, period=self.p.period)

In [11]:
class MyStrategy(bt.Strategy):
    params = dict(period=20)

    def __init__(self):
        sma = btind.SimpleMovingAverage(self.data, period=self.p.period)

In [16]:
class MyStrategy(bt.Strategy):
    params = dict(period=20)

    def __init__(self):
        sma = btind.SimpleMovingAverage(self.data, period=self.p.period)
        print(self.p.period)

# Lines
Again mostly every other object in the platform is a Lines enabled object. From a end user point of view this means:

It can hold one of more line series, being a line series an array of values were the values put together in a chart they would form a line.
A good example of a line (or lineseries) is the line formed by the closing prices of a stock. This is actually a well-known chart representation of the evolution of prices (known as Line on Close)

Regular use of the platform is only concerned with accessing lines. The previous mini-strategy example, lightly extended, comes in handy again:

In [23]:
class MyStrategy(bt.Strategy):
    params = dict(period=20)

    def __init__(self):

        self.movav = btind.SimpleMovingAverage(self.data, period=self.p.period)

    def next(self):
        if self.movav.lines.sma[0] > self.data.lines.close[0]:
            print('Simple Moving Average is greater than the closing price')

Two objects with lines have been exposed:

self.data It has a lines attribute which contains a close attribute in turn

self.movav which is a SimpleMovingAverage indicator It has a lines attribute which contains a sma attribute in turn

It should be obvious from this, that lines are named. They can also be accessed sequentially following the declaration order, but this should only be used in Indicator development

And both lines, namely close and sma can be queried for a point (index 0) to compare the values.

Shorthand access to lines do exist:

xxx.lines can be shortened to xxx.l

xxx.lines.name can be shortened to xxx.lines_name

Complex objects like Strategies and Indicators offer quick access to data’s lines

self.data_name offers a direct access to self.data.lines.name

Which also applies to the numbered data variables: self.data1_name -> self.data1.lines.name

Additionally the line names are directly accessible with:

self.data.close and self.movav.sma

But the notation doesn’t make as clear as the previous one if lines are actually being accessed.

# Lines declaration
If an Indicator is being developed, the lines which the indicator has must be declared.

Just as with params this takes place as a class attribute this time ONLY as a tuple. Dictionaries are not supported because they do not store things following insertion order.


 
For the Simple Moving Average it would be done like this:
```python
class SimpleMovingAverage(Indicator):
    lines = ('sma',)
```
    ...
Note

The comma following the declaration is needed in tuples if you pass a single string to the tuple or else each letter in the string would be interpreted as an item to be added to the tuple. Possibly one of the few spots where Python’s syntax got it wrong.

As seen in the previous example this declaration creates a sma line in the Indicator that can be later accessed in the Strategy’s logic (and possibly by other indicators to create more complex indicators)

For development is sometimes useful to access the lines in a generic non-named manner and this is where numbered access comes in handy:

self.lines[0] points to self.lines.sma
Had more lines been defined they would be accessed with index 1, 2, and higher.

And of course, extra shorthand versions do exist:

self.line points to self.lines[0]

self.lineX point to self.lines[X]

self.line_X point to self.lines[X]

Inside objects which are receiving datas feeds the lines below these data feeds can also be quickly accessed by number:

self.dataY points to self.data.lines[Y]

self.dataX_Y points to self.dataX.lines[X] which is a full shorthard version of self.datas[X].lines[Y]

Accessing lines in Data Feeds
Inside data feeds the lines can also be accessed omitting the lines. This makes it more natural to work with thinks like close prices.

For example: